# Phase Estimation

This tutorial walks through using the following subclasses of `PhaseEstimator`.

* `PhaseEstimation` for using a quantum computer to solve the eigenvalue problem $U |\phi\rangle = \exp(i2\pi \phi) |\phi\rangle$, where $U$ is a unitary operator.
* `IterativePhaseEstimation` for solving the the preceeding problem using an iterative algorithm that uses fewer qubits.
* `HamiltonianPhaseEstimation` for estimating the eigenvalue in $H |\lambda\rangle = \lambda |\lambda\rangle$, where $H$ is a Hermitian operator.

First, we import some of the necessary modules and classes.

In [1]:
import qiskit

from qiskit.algorithms.phase_estimators import (
    PhaseEstimation,
    HamiltonianPhaseEstimation,
    IterativePhaseEstimation,
)

We import some operators that we will use to express the unitaries and prepare the eigenstates.

In [2]:
from qiskit.opflow import H, X, Y, Z, I

## `PhaseEstimation`

We begin with the simplest kind of problem supported by the `PhaseEstimator` subclasses, estimating the phase $\phi$ in the eigenvalue problem

$$U |\phi\rangle = \exp(i2\pi \phi) |\phi\rangle,$$

where $U$ is a unitary operator. All algorithms in this module return an estimate $\phi \in [0,1)$. The algorithm used by `PhaseEstimation` is described in Nielsen and Chuang.
We assume that we can prepare a register in the state $|\phi\rangle$ and that we can implement $U$ in quantum circuits.

For a concrete example, we choose $U = Z$ and $|\phi\rangle = |1\rangle = X|0\rangle$, so that $X$ is the eigenstate-preparation gate. Note that $Z|1\rangle = -|1\rangle$ and $-1 = \exp(i \pi)$, so we expect the algorithm to return $\phi=1/2$.

We will run the example on a quantum simulator, which we prepare like this:

In [3]:
backend = qiskit.Aer.get_backend("statevector_simulator")
qi = qiskit.utils.QuantumInstance(backend=backend)

Now, we set up the phase estimator as an instance of the class `PhaseEstimation`. If the phase can be represented exactly in $t$ bits as $\phi = 0.\phi_1 \ldots \phi_t$, then we need $t$ qubits in the phase estimation register. In the case at hand, $\phi = 1/2$, this means $t=1$. We instantiate the phase estimator like this:

In [4]:
phase_estimator = PhaseEstimation
num_evaluation_qubits=1
phase_est = PhaseEstimation(num_evaluation_qubits=num_evaluation_qubits, quantum_instance=qi)

`PhaseEstimation` requires that circuits that prepare the input state $\phi\rangle$ and implement $U$ be represented as instances of `QuantumCircuit`:

In [5]:
unitary_circuit = Z.to_circuit()
state_preparation = X.to_circuit()

Now we run the phase estimation algorithm and extract just the phase from the returned result object. The phase is equal to $0.5$ as expected.

In [6]:
result = phase_est.estimate(unitary=unitary_circuit, state_preparation=state_preparation)
result.phase

0.5

To get the other eigenvalue of $Z$, corresponding to the problem $Z|0\rangle = |0\rangle$, we need the input state $|0\rangle$. Since $|0\rangle$ is initial state of quantum computers and simulators, we don't need a preparation circuit. In this case we pass `None` for the preparation circuit.

In [7]:
state_preparation = None

We expect a phase $\phi=0$ since $1 = \exp(0)$:

In [8]:
result = phase_est.estimate(unitary=unitary_circuit, state_preparation=state_preparation)
result.phase

0.0

## `IterativePhaseEstimation`

We can also solve the problems above using an alternative algorithm: iterative phase estimation. This is implemented by the class `IterativePhaseEstimation`.

Let's consider again the same problem as above $Z|1\rangle = -|1\rangle$. Because the phase $\phi=1/2$ can be expressed with a single bit, we need a single iteration. We construct the phase estimation object.

In [17]:
num_iterations = 1
iphase_est = IterativePhaseEstimation(num_iterations=num_iterations, quantum_instance=qi)

Recall that the state preparation circuit for this problem is $X$.

In [18]:
state_preparation = X.to_circuit()

result = iphase_est.estimate(unitary=unitary_circuit, state_preparation=state_preparation)
result.phase

0.5

## Estimating eigenvalues of a Hamiltonian

We choose a Hamiltonian that originates in a second-quantized model of the $\text{H}_2$ molecule. The model produces a Hamiltonian with two Fermionic mode. Then, a procedure that maps each mode to a qubit is performed, with the following result:

In [65]:
hamiltonian = (-1.0523732457728587 * (I ^ I)) + (0.3979374248431802 * (I^Z)) \
+ (-0.3979374248431802 * (Z^I)) + (-0.011280104256235324 * (Z^Z)) + (0.18093119978423147 * (X^X))

In general, the state of interest (e.g. the ground state) is not known exactly. Instead, we rely on some other method to prepare an input state that has a significant contribution from the ground state. For the case at hand, such a state may be prepared with the following circuit:

In [70]:
state_preparation = (I^X)

In contrast to the simple examples above, the eigenphase is no longer equal to two raised to a small negative power. In fact, it is not a power of two. The higher the number of evaluation qubits, the better our approximation of the eigenvalues. We choose the following:

In [71]:
num_evaluation_qubits = 6

On a real quantum processor, we might use a trotterization scheme to implement the Hamiltonian. But, since we are running the algorithm on a simulator, we can implement the entire evolution as a single, multi-qubit, gate. This evolution scheme is implemented by the `MatrixEvolution` class.

In [72]:
from qiskit.opflow.evolutions import MatrixEvolution
evolution_type = MatrixEvolution()

Before running the algorithm, let's compute the eigenvalues of the Hamiltonian

In [85]:
import scipy

def eigenvalues(hamiltonian):
    m = hamiltonian.to_matrix()
    return scipy.linalg.eigvals(m)

eigenvalues(hamiltonian).real

array([-1.24458455, -0.88272215, -1.85727503, -0.22491125])

Now we run the algorithm

In [73]:
phase_est = HamiltonianPhaseEstimation(num_evaluation_qubits=num_evaluation_qubits, quantum_instance=qi)
    
result = phase_est.estimate(hamiltonian=hamiltonian, state_preparation=state_preparation,
                            evolution=evolution_type)

Evolved Hamiltonian is not composed of only MatrixOps, converting to Matrix representation, which can be expensive.


The returned value `result` is an intance of the class `HamiltonianPhaseEstimationResult`, which implements simple tools for analysis. The method `filter_phases(cutoff)` returns only those phases whose probability of measurement is greater than `cutoff`. How many phases occur with non-zero probability?

In [82]:
len(result.filter_phases(0.0))  # Passing no argument is the same as 0.0

64

We used 6 phase-evaluation qubits, which corresponds to $2^6 = 64$ phases. So, all of the phases occur with some non-zero probability. Most probabilities are very small, including some that are would be exactly zero if floating point operations were perfect.

It turns out that our input state has overlap with only two eigenstates of the Hamiltonian.
Here, we tune the filter to get just the most important components, which approximate the two eigenvalues.

In [98]:
phase_dict = result.filter_phases(0.01)
phase_dict

{-0.21867555356584834: 0.010899311653025939,
 -1.8551932456759057: 0.9728871781537503}

Determining what value of the cutoff to use is beyond the scope of this tutorial. But, let's look a bit at what is involved. Let's make the cutoff less strict:

In [97]:
phase_dict = result.filter_phases(0.001)
dict(sorted(phase_dict.items(), key=lambda x: -x[1]))  # sort the dict by probability

{-1.8551932456759057: 0.9728871781537503,
 -0.21867555356584834: 0.010899311653025939,
 -1.886070937979869: 0.005092927085890913,
 -1.8243155533719424: 0.0038881471250501775,
 -1.9169486302838326: 0.0011931982782644767,
 -1.793437861067979: 0.001041113814614538}

We know that `-1.886` is not an eigenvalue distinct from, and less than, `-1.855`. But, this module does not provide a way to determine this algorithmically.

You may need to inspect the state of the algorithm before the final result. Before running the phase estimation algorithm, the Hamiltonian is scaled and shifted such that the eigenvalues are mapped to unique phases in $[0, 1)$. Selecting `scaled=False` allows you to see the phases before they are scaled back to the original eigenvalues:

In [99]:
result.filter_phases(0.001, scaled=False)

{0.421875: 0.010899311653025939,
 0.5625: 0.0011931982782644767,
 0.578125: 0.005092927085890913,
 0.59375: 0.9728871781537503,
 0.609375: 0.0038881471250501775,
 0.625: 0.001041113814614538}

The bitstrings are mapped to evenly spaced phases in $[0, 1)$. To see the probabilties of bitstrings measured from the phase register, do the following.

In [102]:
result.filter_phases(0.001, as_float=False, scaled=False)

{'011011': 0.010899311653025939,
 '100100': 0.0011931982782644767,
 '100101': 0.005092927085890913,
 '100110': 0.9728871781537503,
 '100111': 0.0038881471250501775,
 '101000': 0.001041113814614538}

In [105]:
result._phase_estimation_scale._bound

0.9880861537268272

In [106]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright